In [17]:
import pandas as pd
import numpy as np
import psycopg2
import os
from dotenv import load_dotenv
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
import src.soporte_api as sa
import os
from dotenv import load_dotenv

In [18]:
load_dotenv()

True

In [ ]:
#Llamamos a la variable de entorno
DB_NAME = os.getenv("DB_NAME")
DB_USER=os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST= os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")

In [20]:
#Creamos la conexión:
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    host=DB_HOST,
    port=DB_PORT
)

#Creamos el cursor:
cur= conn.cursor()

In [21]:
#Comprobamos que la conexión está bien realizada:
cur.execute("SELECT version();")
cur.fetchone() #para que muestre el resultado de la query anterior

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

# Creación de los datos para las tablas:

In [22]:
#Importo la información del df de reservas:
ARCHIVO_SALIDA = os.getenv("ARCHIVO_SALIDA")
df_reservas = pd.read_pickle(ARCHIVO_SALIDA)
df_reservas.head()

,id_reserva,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,ciudad,id_cliente_unico,valoracion
0,5256cc90-139b-43d2-8ec5-412495d751cf,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-02-25,2025-03-01,2025-03-02,113,268,Novotel Madrid Center,Madrid,C0fc,4.6
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-02-25,2025-03-01,2025-03-02,194,110,ibis budget Madrid Calle 30,Madrid,C1lh,4.4
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,Madrid,C2mc,3.1
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,Madrid,C3tb,3.1
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,Madrid,C4aa,3.1


## 1. Tabla ciudad

In [23]:
#creo el diccionario ciudad
dict_ciudad = {'ciudad':df_reservas.ciudad[0]}
dict_ciudad

{'ciudad': 'Madrid'}

In [24]:
#Creo la query para insertar el valor en la tabla ciudad
ciudad = dict_ciudad.get("ciudad")
insert_query = "INSERT INTO ciudad(nombre_ciudad) VALUES(%s);"
data_to_insert = (ciudad,)
cur.execute(insert_query,data_to_insert)
conn.commit()


In [25]:
def insertar_datos_en_BBDD():
    cur.execute(insert_query,data_to_insert)
    conn.commit()

# 2. Tabla eventos

In [47]:
#Extraigo la información de la api y lo convierto en un df:
ARCHIVO_RAW= os.getenv("ARCHIVO_RAW")
URL_API = os.getenv("URL_API")
ARCHIVO_GUARDAR_EVENTOS_API = os.getenv("ARCHIVO_GUARDAR_EVENTOS_API")
dict_eventos = sa.eventos_api(ARCHIVO_RAW,ARCHIVO_GUARDAR_EVENTOS_API,URL_API)
df_eventos = pd.DataFrame(dict_eventos)
df_eventos.head(10)

,id_evento,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion,ciudad
0,12677513,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,CALLE SAN JUSTO 5,,2025-02-24 00:00:00,2025-03-02 23:59:00,Biblioteca Pública Municipal Iván de Vargas (C...,MADRID
1,12693864,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009,PASEO COLOMBIA 1,,2025-02-27 00:00:00,2025-03-23 23:59:00,Centro Cultural Casa de Vacas (Retiro),MADRID
2,12536930,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Sin información,Sin información,,2024-10-14 00:00:00,2025-06-30 23:59:00,Sin información,MADRID
3,12697332,Almudena Lobera. Secuencia Plano Secuencia,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28014,PLAZA CIBELES 1,,2025-02-13 00:00:00,2025-06-08 23:59:00,CentroCentro,MADRID
4,12513883,Andrea Canepa «Wasi qaymi (Lugar y tacto)»,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28015,CALLE CONDE DUQUE 9,,2025-02-28 00:00:00,2025-05-18 23:59:00,Centro de Cultura Contemporánea Conde Duque,MADRID
5,12425808,Apoyo a la autonomía,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Sin información,Sin información,,2024-09-13 00:00:00,2025-06-20 23:59:00,Sin información,MADRID
6,12411724,Artemisia. La magia de la luna,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,AVENIDA PLANETARIO 16,18:00,2024-07-16 18:00:00,2025-12-28 23:59:00,Planetario de Madrid,MADRID
7,12609463,Artificios teatrales. La escena musical de los...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28015,CALLE AMANIEL 29,,2024-10-16 00:00:00,2025-03-08 23:59:00,Centro Cultural Clara del Rey - Museo ABC (Cen...,MADRID
8,12525792,ArTrics - Laboratorio de Creación 2024/25,http://www.madrid.es/sites/v/index.jsp?vgnextc...,Sin información,Sin información,,2024-10-01 00:00:00,2025-07-31 23:59:00,Sin información,MADRID
9,12682299,Asalto a la lectura. Tres cerditos y más,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28015,CALLE AMANIEL 29,11:00,2025-03-01 11:00:00,2025-03-01 23:59:00,Centro Cultural Clara del Rey - Museo ABC (Cen...,MADRID


In [27]:
#Las claves del diccionario eventos son:
dict_eventos.keys()

dict_keys(['id_evento', 'nombre_evento', 'url_evento', 'codigo_postal', 'direccion', 'horario', 'fecha_inicio', 'fecha_fin', 'organizacion', 'ciudad'])

La tabla eventos tiene que tener los siguientes campos:
- id_evento SERIAL PRIMARY KEY,
- nombre_evento TEXT,
- url_evento TEXT,
- codigo_postal INT,
- direccion TEXT,
- horario TEXT,
- fecha_inicio DATE,
- fecha_fin DATE,
- organizacion TEXT,
- id_ciudad INT REFERENCES ciudad(id_ciudad) ON DELETE CASCADE

In [28]:
#Solo me falta conseguir el id de la ciudad de la BBDD:
cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad")
ciudad_dict=dict(cur.fetchall())
ciudad_dict

{'Madrid': 1}

In [29]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for _,row in df_eventos.iterrows():
    nombre_evento = row["nombre_evento"]
    url_evento = row["url_evento"]
    codigo_postal = row['codigo_postal']
    direccion = row['direccion']
    horario = row['horario']
    fecha_inicio = row['fecha_inicio']
    fecha_fin= row['fecha_fin']
    organizacion= row['organizacion']
    ciudad = row['ciudad'].capitalize()
    id_ciudad = ciudad_dict.get(ciudad)
    data_to_insert.append([nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad])


In [30]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO eventos(nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cur.executemany(insert_query, data_to_insert)
conn.commit()

# 3. Tabla hoteles

La tabla hoteles necesita los siguientes campos: 
- id_hotel SERIAL PRIMARY KEY, -- Puede ser VARCHAR(5)
-  nombre_hotel TEXT,
- competencia BOOL,
- valoracion FLOAT CHECK (valoracion BETWEEN 1 AND 5),
- id_ciudad INT REFERENCES ciudad(id_ciudad) ON DELETE CASCADE

In [31]:
df_reservas.columns

Index(['id_reserva', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'ciudad', 'id_cliente_unico',
       'valoracion'],
      dtype='object')

In [32]:
#Creo la tabla hoteles y elimino los duplicados:
tabla_hoteles = df_reservas[['nombre_hotel', 'competencia', 'valoracion', 'ciudad']]
tabla_hoteles = tabla_hoteles.drop_duplicates()
tabla_hoteles

,nombre_hotel,competencia,valoracion,ciudad
0,Novotel Madrid Center,True,4.6,Madrid
1,ibis budget Madrid Calle 30,True,4.4,Madrid
2,Hotel Monte Verde,False,3.1,Madrid
3,Hotel Brisas del Mar,False,3.1,Madrid
4,Hotel Camino del Sol,False,3.1,Madrid
5,Hotel Puerta del Cielo,False,3.0,Madrid
6,Hotel Encanto Real,False,3.0,Madrid
7,ibis Madrid Centro las Ventas,True,4.5,Madrid
8,Palacio del Sol,False,3.0,Madrid
9,ibis budget Madrid Centro las Ventas,True,4.3,Madrid


In [33]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for _,row in tabla_hoteles.iterrows():
    nombre_hotel = row["nombre_hotel"]
    competencia = row["competencia"]
    valoracion = row['valoracion']
    ciudad = row['ciudad'].capitalize()
    id_ciudad = ciudad_dict.get(ciudad)
    data_to_insert.append([nombre_hotel, competencia, valoracion, id_ciudad])
data_to_insert


[['Novotel Madrid Center', True, 4.6, 1],
 ['ibis budget Madrid Calle 30', True, 4.4, 1],
 ['Hotel Monte Verde', False, 3.1, 1],
 ['Hotel Brisas del Mar', False, 3.1, 1],
 ['Hotel Camino del Sol', False, 3.1, 1],
 ['Hotel Puerta del Cielo', False, 3.0, 1],
 ['Hotel Encanto Real', False, 3.0, 1],
 ['ibis Madrid Centro las Ventas', True, 4.5, 1],
 ['Palacio del Sol', False, 3.0, 1],
 ['ibis budget Madrid Centro las Ventas', True, 4.3, 1],
 ['Hotel Jardines del Rey', False, 2.9, 1],
 ['Novotel Madrid City Las Ventas', True, 4.6, 1],
 ['ibis Madrid Calle Alcalá', True, 4.5, 1],
 ['Hotel Las Estrellas', False, 2.9, 1],
 ['Gran Hotel Madrid', False, 3.1, 1],
 ['ibis budget Madrid Vallecas', True, 4.3, 1],
 ['Hotel Torre Dorada', False, 2.9, 1],
 ['Hotel Palacio Imperial', False, 3.0, 1],
 ['Hotel Luz de Madrid', False, 3.1, 1],
 ['Hotel Los Almendros', False, 3.0, 1],
 ['Hotel Sol y Luna', False, 3.0, 1],
 ['Hotel Mirador Real', False, 3.0, 1],
 ['Pullman Madrid Airport & Feria', True, 4.2, 

In [34]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO hoteles (nombre_hotel,competencia,valoracion,id_ciudad)
VALUES (%s, %s, %s, %s)
"""
cur.executemany(insert_query, data_to_insert)
conn.commit()

# 4. Tabla clientes

La Tabla clientes tiene las siguientes columnas:
- id_cliente VARCHAR(50) PRIMARY KEY,
- nombre TEXT,
- apellido TEXT,
- mail TEXT UNIQUE CHECK (mail LIKE '%@%')

In [35]:
df_reservas.columns

Index(['id_reserva', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'ciudad', 'id_cliente_unico',
       'valoracion'],
      dtype='object')

In [36]:
#Creo la tabla clientes y elimino los duplicados:
tabla_clientes = df_reservas[['id_cliente_unico', 'nombre', 'apellido', 'mail']]
tabla_clientes = tabla_clientes.drop_duplicates()
tabla_clientes

,id_cliente_unico,nombre,apellido,mail
0,C0fc,Feliciana,Cantón,feliciana.cantón@example.com
1,C1lh,Leonardo,Hierro,leonardo.hierro@example.com
2,C2mc,Maite,Calatayud,maite.calatayud@example.com
3,C3tb,Tecla,Bonet,tecla.bonet@example.com
4,C4aa,Amílcar,Andrés,amílcar.andrés@example.com
...,...,...,...,...
14995,C14900lr,Lorena,Román,lorena.román@example.com
14996,C14901tn,Toño,Narváez,toño.narváez@example.com
14997,C14902nv,Nadia,Verdugo,nadia.verdugo@example.com
14998,C14903cm,Custodia,Marco,custodia.marco@example.com


In [37]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for _,row in tabla_clientes.iterrows():
    id_cliente = row["id_cliente_unico"]
    nombre = row["nombre"]
    apellido = row['apellido']
    mail = row['mail']
    data_to_insert.append([id_cliente, nombre, apellido, mail])
len(data_to_insert)

14905

In [38]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO clientes (id_cliente, nombre, apellido, mail)
VALUES (%s, %s, %s, %s)
"""
cur.executemany(insert_query, data_to_insert)
conn.commit()

# 5. Tabla reservas

La tabla reserva tiene las siguientes columnas: 
- id_reserva VARCHAR(50) PRIMARY KEY,
- fecha_reserva DATE,
- inicio_estancia DATE,
- final_estancia DATE,
- precio_noche FLOAT CHECK (precio_noche >= 0),
- id_cliente VARCHAR(50) REFERENCES clientes(id_cliente) ON DELETE CASCADE,
- id_hotel INT REFERENCES hoteles(id_hotel) ON DELETE CASCADE

In [39]:
df_reservas.columns

Index(['id_reserva', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'ciudad', 'id_cliente_unico',
       'valoracion'],
      dtype='object')

In [40]:
#compruebo que el id_reserva es unico
df_reservas['id_reserva'].nunique() == df_reservas.shape[0]

True

In [41]:
#Creo la tabla reservas:
tabla_reservas = df_reservas[['id_reserva', 'fecha_reserva', 'inicio_estancia', 'final_estancia', 'precio_noche', 'id_cliente_unico', 'nombre_hotel' ]]
tabla_reservas = tabla_reservas.drop_duplicates()
tabla_reservas.head()

,id_reserva,fecha_reserva,inicio_estancia,final_estancia,precio_noche,id_cliente_unico,nombre_hotel
0,5256cc90-139b-43d2-8ec5-412495d751cf,2025-02-25,2025-03-01,2025-03-02,268,C0fc,Novotel Madrid Center
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,2025-02-25,2025-03-01,2025-03-02,110,C1lh,ibis budget Madrid Calle 30
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,2025-02-09,2025-03-01,2025-03-02,119.96,C2mc,Hotel Monte Verde
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,2025-02-08,2025-03-01,2025-03-02,211.53,C3tb,Hotel Brisas del Mar
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,2025-02-02,2025-03-01,2025-03-02,378.37,C4aa,Hotel Camino del Sol


In [42]:
conn.rollback()

In [43]:
#Obtengo el id del hotel a partir de la BBDD:
cur.execute("SELECT nombre_hotel, id_hotel FROM hoteles")
hotel_dict=dict(cur.fetchall())

In [44]:
#Creo los datos para insertarlos en la BBDD:
data_to_insert=[]
for _,row in tabla_reservas.iterrows():
    id_reserva = row["id_reserva"]
    fecha_reserva = row["fecha_reserva"]
    inicio_estancia = row['inicio_estancia']
    final_estancia = row['final_estancia']
    precio_noche = row['precio_noche']
    id_cliente = row['id_cliente_unico']
    nombre_hotel = row['nombre_hotel']
    id_hotel= hotel_dict.get(nombre_hotel)
    data_to_insert.append([id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel])
len(data_to_insert)

15000

In [45]:
#Subo la información a la BBDD:
insert_query = """
INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
cur.executemany(insert_query, data_to_insert)
conn.commit()

# 6. Cierro la conexión y el cursor

In [48]:
cur.close() #cierro el cursor
conn.close() #cierro la conexión